In [2]:
import numpy as np

In [3]:
#Conversions
u_to_kg = 1.66054e-27 #amu to kg
H_to_kJ_mol = 2625.5  #Hartree to kJ/mol
H_to_J = 4.35974e-18 #J

#masses
li_m = 6.941*u_to_kg #kg
h_m = 1.00784*u_to_kg #kg

#Constants
h = 6.62607004e-34 #J*s
hbar = 1.0545718e-34 #J*s
k_B = 1.380649e-23 #J/K

In [4]:
#calculate reduced mass for diatomic molecule
def reduced_mass(m_1,m_2):
    mu_inv = (1/m_1) + (1/m_2)
    return mu_inv**-1

In [49]:
#calculate fundamental vibrational energy in Hz
def vib_freq(D_e,mu,alpha):
    D_e = D_e*H_to_kJ_mol*10**3
    nu = (alpha/(2*np.pi))*np.sqrt((2*D_e)/mu)
    return nu

In [50]:
#Vibrational energy Levels
def vib_eLevels(n,nu,D_e):
    term_1 = h*nu*(n+(1/2))
    term_2 = term_1**2/(4*D_e)
    return term_1 - term_2

In [51]:
#Rotational energy levels:
def rot_eLevels(J,r_0,mu):
    term = (hbar**2/(2*I_m))*(J*(J+1))
    return term

In [57]:
#From fit:
D_e = 0.24465576367809916 #Hartree
alpha = 1.0814331538285966e10 #sqrt(mol)/Angstr.
r_0 = 1.4638508752003854e-10 #angstrom

m1 = li_m
m2 = h_m

mu = reduced_mass(m1,m2)
nu_0 = vib_freq(D_e,mu,alpha)
I_m = r_0**2*mu

n_max = int((2*D_e*H_to_J - h*nu_0)/(h*nu_0)) #max vibrational mode
E_diss = D_e*H_to_J - (vib_eLevels(n_max,nu_0,D_e*H_to_kJ_mol))
J_max = int((np.sqrt(hbar**2+8*E_diss*r_0**2*mu)- hbar)/(2*hbar)) #max rotational mode

In [ ]:
#Partion Function Components
def q_trans(T,V):
    return ((2*np.pi*mu*k_B*T)/h**2)**(3/2)*V

dq_trans = lambda T: (3/2)*(hbar**2/(2*np.pi*mu*k_B))*(1/T)

def q_vib(n_max,T):
    modes = np.empty(len(T))
    terms = []
    for j in range(len(T)):
        for i in range(n_max):
            terms.append(np.exp(-vib_eLevels(i,nu_0,D_e)/(k_B*T[j])))
        modes[j] = np.sum(terms)
        terms = []
    return modes

def q_rot(J_max,T):
    theta_r = hbar**2/(2*I_m*k_B)
    modes = np.empty(len(T))
    terms = []
    for i in range(len(T)):
        for J in range(J_max):
            terms.append(((2*J+1)*np.exp((-theta_2*J*(J+1))/T[i])))
        modes[i] = np.sum(terms)
        terms = []
    return modes

#take derivative of log of partition function
def derivative_log(func,T,i_max):
    y = np.log(func(i_max,T))
    dy_c = (y[2:] - y[:-2]) / (T[2:] - T[:-2]) #central difference method
    return dy_c

In [ ]:
def internal_E(T):
    deriv = dq_trans(T) + derivative_log(q_vib,T,n_max) + derivative_log(q_rot,T,J_max)
    values = np.empty(len(T))
    for i in range(len(values)):
        values[i] = k_B*T[i]**2*deriv[i]
    return values

def helmholtz(T,V):
    values = np.empty(len(T))
    for i in range(len(values)):
        values[i] = np.log(q_trans(T[i],V[i])) + np.log(q_vib(n_max,T))[i] + np.log(q_rot(J_max,T))[i]
    return values

In [ ]:
def enthalpy(T):
    P_V = k_B*T
    values = np.empty(len(T))
    for i in range(len(values)):
        values[i] = internal_E(T)[i] + P_V[i]
    return values

def entropy(T,V):
    values = np.empty(len(T))
    for i in range(len(values)):
        values[i] = (internal_E(T)[i] - helmholtz(T,V)[i]) / T[i]
    return values